# This notebook scans all directories in an input bucket and converts all files to wav

This occurs inplace and deletes the original file and replaces it with a .wav file, It overwrites an existing file by the same name.

It expects input of a "bucket" folder, consisting of subfolders named as species, and the audio files within.
If converting a single folder, create a parent folder to put it into.

Audio file formats are
- .mp3
- .flac
- .wav

Video file inputs are
- .mp4
- .m4a

In [76]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import librosa, librosa.display
import pathlib
import glob
import pathlib
import subprocess

In [77]:
# input the bucket directory here
# must be a parent directory with folders of audio files
bucket ="/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1"
species_bucket = os.listdir(bucket)
species_bucket

['.DS_Store', 'Carhorn', 'tiger Quoll', 'eBird files 2M and smaller']

In [78]:
# writes creates a dataframe of species (folder names)

species_bucket = pd.DataFrame(species_bucket, columns = ["animal species"])
species_bucket.index = np.arange(1, len(species_bucket) + 1)
species_bucket = species_bucket.sort_values("animal species")
species_bucket = species_bucket.iloc[1:]
species_bucket.index = np.arange(1, len(species_bucket) + 1)
print("files: {}".format(len(species_bucket)))
species_bucket.head()

files: 3


,animal species
1,Carhorn
2,eBird files 2M and smaller
3,tiger Quoll


In [79]:
## extract species as dataframe
# function to crearte a dataframe of all species (folders) in the bucket, listed alphabetically
def species_as_dataframe(folder_filepath):
    species_list = os.listdir(folder_filepath)
    species_dataframe = pd.DataFrame(species_list, columns = ["animal species"])
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    species_dataframe = species_dataframe.sort_values("animal species")
    species_dataframe = species_dataframe.iloc[1:]
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    return species_dataframe 

species_as_dataframe(bucket)


,animal species
1,Carhorn
2,eBird files 2M and smaller
3,tiger Quoll


In [80]:
# function to crearte a dataframe of full directory names of species in the bucket, listed alphabetically

def file_paths_as_df(bucket):  # maybe change to receive the column value too
    bucket_folders = glob.glob(bucket+"/*")
    bucket_folders = pd.DataFrame(bucket_folders, columns = ["animal species folder paths"])
    bucket_folders = bucket_folders.sort_values("animal species folder paths")
    bucket_folders.index = np.arange(1, len(bucket_folders) + 1)
    bucket_folders.head()
    return bucket_folders
bucket_file_paths_as_dataframe = file_paths_as_df(bucket)
bucket_file_paths_as_dataframe

,animal species folder paths
1,/Volumes/UGREEN Samsung SSD/project echo audio...
2,/Volumes/UGREEN Samsung SSD/project echo audio...
3,/Volumes/UGREEN Samsung SSD/project echo audio...


In [81]:
# this takes a dataframe and changes it back to list
def file_paths_as_list(bucket_folders_dataframe):
    bucket_folders_list = bucket_folders_dataframe.values.tolist()
    bucket_folders_list = list(np.concatenate(bucket_folders_list).flat)
    bucket_folders_list[0:1]
    return bucket_folders_list

In [82]:
bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket))
bucket_file_paths_as_list

['/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1/Carhorn',
 '/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1/eBird files 2M and smaller',
 '/Volumes/UGREEN Samsung SSD/project echo audio/test data/test_1/tiger Quoll']

In [83]:
# this creates a .wav version of a file past to it
# it will save with the same name
# it will overwrite if an existing .wav file exists with the same name

def convert_individual_file(file_path):
    parent_path = pathlib.PurePath(file_path).parent    
    file_name = pathlib.Path(file_path).stem   
    output_file_path = str(parent_path) + "/" + file_name + ".mp3"
    # write the new wav file forcing overwrite 
    subprocess.call(['ffmpeg', '-y', '-i', file_path,
                 output_file_path])
    return

In [84]:
# this function checks the filepath passed to it and checks the filetype
# if it is one of the listed filetypes ot be converted, it will make a new version 
# as a .wav and delete the original

def check_for_conversion(i, species_filepath):  
    audio_file_paths = glob.glob(species_filepath+"/*")

    for i in range(len(audio_file_paths)):
        if pathlib.Path(audio_file_paths[i]).suffix == ".wav":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".flac":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".m4a":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".mp4":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])
    return

In [85]:
### working here ###

'''
pathlib.PurePath would work much better here I'm sure ...
'''

def convert_files(bucket_file_path):
    print("Folders scanned")
    print(".........")
    for i in range(len(species_bucket)):
        bucket_species_dataframe = species_as_dataframe(bucket_file_path)
        bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket_file_path))
        current_species_file_pathame = bucket_file_paths_as_list[i]
        check_for_conversion(i, current_species_file_pathame)
    print(bucket_species_dataframe)
    print(".........")
    print("Conversion Complete")




In [86]:
convert_files(bucket)

Folders scanned
.........


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disab

               animal species
1                     Carhorn
2  eBird files 2M and smaller
3                 tiger Quoll
.........
Conversion Complete


size=     961kB time=00:01:01.44 bitrate= 128.1kbits/s speed= 127x    
video:0kB audio:961kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.034356%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-fre

In [ ]:
# run a second time to generate a clean output
convert_files(bucket)